In [1]:
import tracemalloc
import memory_profiler as mp
# Doc du lieu tu file dau vao
import numpy as np
'''def read_input_irp(Testcase.txt.txt):'''
tracemalloc.start()

with open("../testcase/test.txt", "r") as file_it:
    lines = file_it.readline().strip().split()
    nb_customers = int(lines[0])-1
    time_periods = int(lines[1])
    capacity = int(lines[2])
    nb_vehicles = int(lines[3])
    start_level = [None] * (nb_customers+1)
    max_level = [None] * (nb_customers+1)
    min_level = [None] * (nb_customers+1)
    daily_consumption = [None] * (nb_customers+1)
    inventory_cost = [None] * (nb_customers+1)
    lines = file_it.readline().strip().split()
    start_level_supplier = int(lines[1])
    production_rate_supplier = int(lines[2])
    holding_cost_supplier = float((lines[3]))
    for i in range(nb_customers):
        lines = file_it.readline().strip().split()
        j = int(lines[0])
        start_level[j] = int(lines[1])
        max_level[j] = int(lines[2])
        min_level[j] = int(lines[3])
        daily_consumption[j] = int(lines[4])
        inventory_cost[j] = float(lines[5])
    distance_matrix = np.zeros((nb_customers+1, nb_customers+1))
    for i in range(nb_customers+1):
        lines = file_it.readline().strip().split()
        for j in range(nb_customers+1):
            distance_matrix[i][j] = float(lines[j])
distance_customer = distance_matrix[1:,1:]
sort_dis=[[0 for i in range(len(distance_matrix))] for j in range(len(distance_matrix))]
for i in range(len(distance_matrix)):
    sort_dis[i] = distance_matrix[i].copy()
    sort_dis[i].sort()
print(1)
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

1
Current memory usage is 0.006748MB; Peak was 0.048221MB


In [2]:
#Thuc hien phan cum cac diem trong du lieu
from sklearn.cluster import AgglomerativeClustering

clustering = AgglomerativeClustering(n_clusters=nb_vehicles, metric='precomputed',linkage='complete')
clusters = clustering.fit_predict(distance_customer)

indices_label = []
sum = [0]*(nb_vehicles)
for i in range(nb_vehicles):
    list_label = np.where(clusters == i)[0]
    list_label = [value + 1 for value in list_label]
    indices_label.append(list_label)
    sum[i]=0
    for index in list_label:
        sum[i]+=daily_consumption[index]
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 38.337384MB; Peak was 38.452304MB


In [3]:
#Ham di chuyen diem sang cum khac phu hop hon
def Switch(Index):
    global sum
    i_index = 0
    j_index = 0
    a = 0
    b = 0
    while sum[Index] > capacity:
        maxDis = 0
        for i in indices_label[Index]:
            for j in indices_label[Index]:
                if distance_matrix[i][j] > maxDis:
                    maxDis = distance_matrix[i][j]
                    a=i
                    b=j
        Min = distance_matrix[a][1]
        i_index = nb_vehicles+1
        for i in range(1,len(distance_matrix)):
            if i not in indices_label[Index]:
                index_clus = clusters[i-1]
                if Min < distance_matrix[a][i] and sum[index_clus] + daily_consumption[a] < capacity:
                    Min = distance_matrix[a][i]
                    i_index = i
        Min = distance_matrix[b][1]
        j_index = nb_vehicles+1
        for i in range(1,len(distance_matrix)):
            if i not in indices_label[Index]:
                index_clus = clusters[i-1]
                if Min < distance_matrix[b][i] and sum[index_clus] + daily_consumption[b] < capacity:
                    Min = distance_matrix[b][i]
                    j_index = i
        if j_index == nb_vehicles+1:
            indices_label[Index].remove(a)
            sum[Index]  = sum[Index] - daily_consumption[a]
            indices_label[clusters[i_index-1]].append(a)
            sum[clusters[i_index-1]] += daily_consumption[a]
        elif i_index == nb_vehicles+1:
            indices_label[Index].remove(b)
            sum[Index]  = sum[Index] - daily_consumption[b]
            indices_label[clusters[j_index-1]].append(b)
            sum[clusters[j_index-1]] += daily_consumption[b]
        else:
            if distance_matrix[i_index][a] < distance_matrix[j_index][b]:
                indices_label[Index].remove(a)
                sum[Index]  = sum[Index] - daily_consumption[a]
                indices_label[clusters[i_index-1]].append(a)
                sum[clusters[i_index-1]] += daily_consumption[a]
            else:
                indices_label[Index].remove(b)
                sum[Index]  = sum[Index] - daily_consumption[b]
                indices_label[clusters[j_index-1]].append(b)
                sum[clusters[j_index-1]] += daily_consumption[b]

In [4]:
#Kiem tra cac cum xem co cum nao co luong hang can van chuyen vuot qua capacity cua phuong tien thi tien hanh chuyen 1 vai vi tri trong cum sang cum khac
for i in range(nb_vehicles):
    if(sum[i]>capacity):
        Switch(i)
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 38.134854MB; Peak was 38.52433MB


In [5]:
#Tien hanh dinh tuyen duong di 
def Routing(cluster):
    routing = []
    for k in range(len(cluster)):
        clone = cluster[k].copy()
        a = [0]
        m = 0
        while clone:
            i=clone[0]
            m = distance_matrix[a[-1]][clone[0]]
            for location in clone:
                if distance_matrix[a[-1]][location] < m:
                    m = distance_matrix[a[-1]][location]
                    i = location      
            clone.remove(i)
            a.append(i)
        a.append(0)
        for i in range(1,len(a)-3):
            if distance_matrix[a[i-1]][a[i]] + distance_matrix[a[i+2]][a[i+3]] > distance_matrix[a[i-1]][a[i+2]] + distance_matrix[a[i]][a[i+3]]:
                tmp = a[i]
                a[i] = a[i+2]
                a[i+2]=tmp
        routing.append(a)
    
    return routing


In [6]:
#Luu tuyen duong di vao routing
routing = Routing(indices_label)
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 38.453195MB; Peak was 38.562232MB


In [7]:
#sap xep cac dia diem theo thu tu tang dan cua chi phi ton kho
inventory_cost[0] = 0
sorted_cost = sorted(enumerate(inventory_cost), key=lambda x: x[1])
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 38.318442MB; Peak was 38.562232MB


In [8]:
#Ham them dia diem vao chuyen di
def insertRout(rout, value,indexClus):
    global routing
    i = j = 0
    check = 1
    if len(rout)==0:
        rout.append(value)
    else:
        while check:
            if routing[indexClus][j] != value:
                if rout[i] == routing[indexClus][j]:
                    i+=1
                    if i == len(rout):
                        rout.append(value)
                        break
            else:
                rout.insert(i,value)
                break
            j+=1
    return rout

In [9]:
#Ham tinh toan chi phi van chuyen
def cost_cal(trip):
    cost = 0
    for i in range(len(trip)):
        for j in range(len(trip[i])):
            if j>0:
                cost+=distance_matrix[trip[i][j-1]][trip[i][j]]
            else:
                cost+=distance_matrix[0][trip[i][j]]
        if(trip[i]):
            cost+=distance_matrix[0][trip[i][j]]
    return cost        

In [10]:
#Ham tim xem dia diem nay thuoc chuyen di cua phuong tien nao
def findCluster(value):
    global routing
    for i in range(len(routing)):
        if value in routing[i]:
            return i

In [11]:
#thuc hien tinh toan cac chi phi
trans = [[0 for _ in range(nb_customers+1)] for _ in range(time_periods)]
rout = [[[] for _ in range(nb_vehicles+1)] for _ in range(time_periods)]
cap = [0 for _ in range(nb_vehicles)]
transportation_cost=0
inventory_cost_customer=0
inventory_cost_depot=0
TotalPro = start_level_supplier
for time in range(time_periods):
    TotalPro = TotalPro + production_rate_supplier
    for nb in range(nb_vehicles):
        cap[nb] = capacity
        for lct in routing[nb][1:len(routing[nb])-1]:
            if(start_level[lct] - daily_consumption[lct]>=min_level[lct]):
                start_level[lct] = start_level[lct] - daily_consumption[lct]
                continue
            else:
                trans[time][lct] = daily_consumption[lct] + min_level[lct] - start_level[lct]
                if TotalPro > trans[time][lct]:
                    TotalPro = TotalPro - trans[time][lct]
                else:
                    trans[time][lct] = TotalPro
                    TotalPro = 0
                start_level[lct] = start_level[lct] + trans[time][lct] - daily_consumption[lct]
                cap[nb] -= trans[time][lct]
                rout[time][nb].append(lct)
    for location in sorted_cost[1:]:
        if location[1]<holding_cost_supplier:
            x = findCluster(location[0])
            if cap[x]>0 and TotalPro>0 and max_level[lct]-start_level[lct]>0:
                count = min(cap[x],min(TotalPro,max_level[location[0]]-start_level[location[0]]))
                cap[x]-=count
                TotalPro-=count
                start_level[location[0]]+=count
                trans[time][location[0]]+=count
                rout[time][x] = insertRout(rout[time][x], location[0],x)
    transportation_cost += cost_cal(rout[time])
    inventory_cost_depot += TotalPro * holding_cost_supplier
    for i in range(1,nb_customers+1):
        inventory_cost_customer += start_level[i] * inventory_cost[i]
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 38.592493MB; Peak was 38.735671MB


In [12]:
for time in range(time_periods):
    print('Month '+str(time+1))
    for i in range(nb_vehicles):
        print('Rout '+str(i+1)+': ',end=' ')
        print('0 -',end= ' ')
        for lct in rout[time][i]:
            print(str(lct) + ' ( ' + str(trans[time][lct]) + ' ) -', end = ' ')
        print(str(0)+'\n')

"for time in range(time_periods):\n    print('Week '+str(time+1))\n    for i in range(nb_vehicles):\n        print('Rout '+str(i+1)+': ',end=' ')\n        print('0 -',end= ' ')\n        for lct in rout[time][i]:\n            print(str(lct) + ' ( ' + str(trans[time][lct]) + ' ) -', end = ' ')\n        print(str(0)+'\n')\n"

In [14]:
print(round(transportation_cost,2))
print(round(inventory_cost_depot,2))
print(round(inventory_cost_customer,2))
print(round(transportation_cost+inventory_cost_depot+inventory_cost_customer,2))
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

1905.0
51.21
6.8
1963.01
Current memory usage is 38.592493MB; Peak was 38.735671MB
